## Hyperparam Tuning

Now that we know which models are performing better, it's time to perform cross validation and tune hyperparameters.
- Do a google search for hyperparameter ranges for each type of model.

GridSearch/RandomSearch are a great methods for checking off both of these tasks.

There is a fairly significant issue with this approach for this particular problem (described below). But in the interest of creating a basic functional pipeline, you can just use the default Sklearn methods for now.

## Preventing Data Leakage in Tuning - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it, if you have time!**

BUT we have a problem - if we calculated a numerical value to encode city (such as the mean of sale prices in that city) on the training data, we can't cross validate 
- The rows in each validation fold were part of the original calculation of the mean for that city - that means we're leaking information!
- While sklearn's built in functions are extremely useful, sometimes it is necessary to do things ourselves

You need to create two functions to replicate what Gridsearch does under the hood. This is a challenging, real world data problem! To help you out, we've created some psuedocode and docstrings to get you started. 

**`custom_cross_validation()`**
- Should take the training data, and divide it into multiple train/validation splits. 
- Look into `sklearn.model_selection.KFold` to accomplish this - the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) shows how to split a dataframe and loop through the indexes of your split data. 
- Within your function, you should compute the city means on the training folds just like you did in Notebook 1 - you may have to re-join the city column to do this - and then join these values to the validation fold

This psuedocode may help you fill in the function:

```python
kfold = KFold() # fit sklearn k folds on X_train
train_folds = []
val_folds = []
for training_index, val_index in kfold.split(X_train):
    train_fold, val_fold = #.iloc loop variables on X_train

    # recompute training city means like you did in notebook 1 
    # merge to validation fold
        
    train_folds.append(train_fold)
    val_folds.append(val_fold)

    return train_folds, val_folds
```


**`hyperparameter_search()`**
- Should take the validation and training splits from your previous function, along with your dictionary of hyperparameter values
- For each set of hyperparameter values, fit your chosen model on each set of training folds, and take the average of your chosen scoring metric. [itertools.product()](https://docs.python.org/3/library/itertools.html) will be helpful for looping through all combinations of hyperparameter values
- Your function should output the hyperparameter values corresponding the highest average score across all folds. Alternatively, it could also output a model object fit on the full training dataset with these parameters.


This psuedocode may help you fill in the function:

```python
hyperparams = # Generate hyperparam options with itertools
hyperparam-scores = []
for hyperparam-combo in hyperparams:

    scores = []

    for folds in allmyfolds:
        # score fold the fold with the model/ hyperparams
        scores.append(score-fold)
        
    score = scores.mean()
    hyperparam-scores.append(score)
# After loop, find max of hyperparam-scores. Best params are at same index in `hyperparams` loop iteratble
```

Docstrings have been provided below to get you started. Once you're done developing your functions, you should move them to `functions_variables.py` to keep your notebook clean 

Bear in mind that these instructions are just one way to tackle this problem - the inputs and output formats don't need to be exactly as specified here.

In [38]:
# develop your custom functions here

def custom_cross_validation(training_data, n_splits =5):
    '''creates n_splits sets of training and validation folds

    Args:
      training_data: the dataframe of features and target to be divided into folds
      n_splits: the number of sets of folds to be created

    Returns:
      A tuple of lists, where the first index is a list of the training folds, 
      and the second the corresponding validation fold

    Example:
        >>> output = custom_cross_validation(train_df, n_splits = 10)
        >>> output[0][0] # The first training fold
        >>> output[1][0] # The first validation fold
        >>> output[0][1] # The second training fold
        >>> output[1][1] # The second validation fold... etc.
    '''

    return training_folds, validation_folds

def hyperparameter_search(training_folds, validation_folds, param_grid):
    '''outputs the best combination of hyperparameter settings in the param grid, 
    given the training and validation folds

    Args:
      training_folds: the list of training fold dataframes
      validation_folds: the list of validation fold dataframes
      param_grid: the dictionary of possible hyperparameter values for the chosen model

    Returns:
      A list of the best hyperparameter settings based on the chosen metric

    Example:
        >>> param_grid = {
          'max_depth': [None, 10, 20, 30],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 4],
          'max_features': ['sqrt', 'log2']} # for random forest
        >>> hyperparameter_search(output[0], output[1], param_grid = param_grid) 
        # assuming 'ouput' is the output of custom_cross_validation()
        [20, 5, 2, 'log2'] # hyperparams in order
    '''

    return hyperparameters


## Hyperparam Tuning

In [39]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [40]:
# initiate models + parameter slots
ridge = Ridge(random_state=42)
svm = SVR()
forest = RandomForestRegressor(random_state=42)
boost = XGBRegressor(objective='reg:squarederror',random_state=42)

In [41]:
import pickle
import pandas as pd

# import data
get_Xtrain = open("pickles/processed/X_train_scaled.txt", "rb")
X_train = pickle.load(get_Xtrain)

get_Xtest = open("pickles/processed/X_test_scaled.txt", "rb")
X_test = pickle.load(get_Xtest)

get_ytrain = open("pickles/processed/y_train_cleaned.txt", "rb")
y_train = pickle.load(get_ytrain)

get_ytest = open("pickles/processed/y_test_cleaned.txt", "rb")
y_test = pickle.load(get_ytest)

In [42]:
# ridge parameters to tune
space_ridge = {
    'alpha': [10,50,100,150,200]
    }

In [43]:
# ridge grid search
search_ridge = GridSearchCV(ridge, space_ridge, scoring='r2')
result_ridge = search_ridge.fit(X_train,y_train.values)
print('Best Score: %s' % result_ridge.best_score_)
print('Best Hyperparameters: %s' % result_ridge.best_params_)

Best Score: 0.6140723555661574
Best Hyperparameters: {'alpha': 150}


In [ ]:
# svm parameters to tune
space_svm = {
    'kernel': ['rbf','linear','polynomial'],
    'C': [10000,50000],
    'epsilon': [0.1,1,1.1]
    }

note: C is cut off based on where returns become miniscule; it would keep going up forever if I let it. 

In [45]:
# svm grid search
search_svm = GridSearchCV(svm, space_svm, scoring='r2')
result_svm = search_svm.fit(X_train,y_train.values)
print('Best Score: %s' % result_svm.best_score_)
print('Best Hyperparameters: %s' % result_svm.best_params_)

Best Score: 0.5701299857512059
Best Hyperparameters: {'C': 50000, 'epsilon': 0.001, 'kernel': 'linear'}


In [46]:
# random forest parameters to tune
space_forest = {
    'n_estimators': [350,400,450],
    'max_depth': [18,20,22]
    }

In [47]:
# random forest grid search
search_forest = GridSearchCV(forest, space_forest, scoring='r2')
result_forest = search_forest.fit(X_train,y_train.values)
print('Best Score: %s' % result_forest.best_score_)
print('Best Hyperparameters: %s' % result_forest.best_params_)

Best Score: 0.6524860785593051
Best Hyperparameters: {'max_depth': 20, 'n_estimators': 400}


In [48]:
# xgboost parameters to tune
space_boost = {
    'alpha': [1.5,2,2.5],
    'lambda': [75,100,125],
    'max_depth': [6,7,8],
    'learning_rate': [0.5,0.4,0.3],
    'n_estimators': [40,45,50]
    }

In [49]:
# xgboost grid search
search_boost = GridSearchCV(boost, space_boost,scoring='r2')
results_boost = search_boost.fit(X_train,y_train.values)
print('Best Score: %s' % results_boost.best_score_)
print('Best Hyperparameters: %s' % results_boost.best_params_)

Best Score: 0.6311209872956087
Best Hyperparameters: {'alpha': 2, 'lambda': 100, 'learning_rate': 0.4, 'max_depth': 7, 'n_estimators': 45}


We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

In [ ]:
# initiate best models
ridge_best = Ridge(alpha=150,random_state=42)
svm_best = SVR(kernel='linear',C=50000,epsilon=1)
forest_best = RandomForestRegressor(n_estimators=400,max_depth=20,random_state=42)
boost_best = XGBRegressor(objective='reg:squarederror',learning_rate=0.4,max_depth=7,n_estimators=45,reg_alpha=2,reg_lambda=100,
random_state=42)

In [51]:
y_test.shape

(343,)

In [52]:
X_test.shape

(343, 46)

In [53]:
# run optimized ridge model
ridge_best.fit(X_train,y_train.values)
y_pred_ridge = ridge_best.predict(X_test)

ridge_mse = mean_squared_error(y_test.values,y_pred_ridge)
r2_ridge = r2_score(y_test.values,y_pred_ridge)

# print results
print("Ridge regression model:")
print(f"Mean Squared Error: {ridge_mse}")
print(f"R-squarred: {r2_ridge}")

Ridge regression model:
Mean Squared Error: 151113312121.77924
R-squarred: 0.4885892897226519


In [54]:
# run optimized svm model
svm_best.fit(X_train,y_train.values)
y_pred_svm = svm_best.predict(X_test)

svm_mse = mean_squared_error(y_test.values,y_pred_svm)
r2_svm = r2_score(y_test.values,y_pred_svm)

# print results
print("SVM model:")
print(f"Mean Squared Error: {svm_mse}")
print(f"R-squarred: {r2_svm}")

SVM model:
Mean Squared Error: 179803222483.83557
R-squarred: 0.3914944194558383


In [55]:
# run optimized random forest model
forest_best.fit(X_train,y_train.values)
y_pred_forest = forest_best.predict(X_test)

forest_mse = mean_squared_error(y_test.values,y_pred_forest)
r2_forest = r2_score(y_test.values,y_pred_forest)

# print results
print("Random forest model:")
print(f"Mean Squared Error: {forest_mse}")
print(f"R-squarred: {r2_forest}")

Random forest model:
Mean Squared Error: 112273353520.7802
R-squarred: 0.620034829075746


In [56]:
# run optimized xgboost model
boost_best.fit(X_train,y_train.values)
y_pred_boost = boost_best.predict(X_test)

boost_mse = mean_squared_error(y_test.values,y_pred_boost)
r2_boost = r2_score(y_test.values,y_pred_boost)

# print results
print("XGBoost model:")
print(f"Mean Squared Error: {boost_mse}")
print(f"R-squarred: {r2_boost}")

XGBoost model:
Mean Squared Error: 157566505561.34885
R-squarred: 0.4667498356457813


In [57]:
# save models as pickles
with open('pickles/models/ridge_best.txt', 'wb') as fh:
   pickle.dump(ridge_best, fh)

with open('pickles/models/svm_best.txt', 'wb') as fh:
   pickle.dump(svm_best, fh)

with open('pickles/models/forest_best.txt', 'wb') as fh:
   pickle.dump(forest_best, fh)

boost_best.save_model('pickles/models/boost_best.json')

## Building a Pipeline (Stretch)

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it if you have time!**

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [58]:
# Build pipeline here

Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [59]:
# save your pipeline here